# Initial Imports

In [1]:
import os
import sys

# Get the current working directory (where the Jupyter Notebook is being executed)
current_file_dir = os.getcwd()  # This works in Jupyter Notebooks

# Add the subdirectories to sys.path
data_path = os.path.join(current_file_dir, '../data')  # Removed '../' since it's relative to cwd
python_path = os.path.join(current_file_dir, '../python')

sys.path.append(data_path)
sys.path.append(python_path)

# Importing modeuls from the python subdirectory
from instrument_dictionaries import generate_system_mueller_matrix, \
    update_systemMM, generate_measurement, parse_configuration, perform_optimization
import helper_funcs

# Testing perform_optimization function

In [2]:
import numpy as np
from scipy.optimize import minimize

# Using previous fit
previous_fit = np.load("../data/scipy_minimize_20230914_675nm_restrictive_HWP_and_IMR.npy")

# Starting guess values
theta_pol = previous_fit[0]
delta_HWP = previous_fit[1] # Add your actual delta_HWP value
offset_HWP = previous_fit[2] # Add your actual offset_HWP value
delta_derot = previous_fit[3] # Add your actual delta_derot value
offset_derot = previous_fit[4] # Add your actual offset_derot value
delta_opts = previous_fit[5] # Add your actual delta_opts value
epsilon_opts = previous_fit[6] # Add your actual epsilon_opts value
rot_opts = previous_fit[7] # Add your actual rot_opts value
delta_FLC = previous_fit[8] # Add your actual delta_FLC value
rot_FLC = previous_fit[9] # Add your actual rot_FLC value
em_gain = previous_fit[10] # Add your actual em_gain value

# Define the instrument configuration as a system dictionary
system_dict = {
    "components": {
        "parang_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "m3": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * 0, "epsilon": 0.03},
            "tag": "on-sky",
        },
        "alt_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "lp": {
            "type": "LinearPolarizer",
            "properties": {"theta": theta_pol},
            "tag": "internal",
        },
        "hwp": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * delta_HWP, "delta_theta": offset_HWP},
            "tag": "internal",
        },
        "image_rotator": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * delta_derot, "delta_theta": offset_derot},
            "tag": "internal",
        },
        "optics": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * rot_opts, "epsilon": epsilon_opts},
            "tag": "on-sky",
        },
        "flc": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * delta_FLC, "theta": rot_FLC},
            "tag": "internal",
        },
        "dichroic": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0, "theta": 0},
            "tag": "internal",
        },
        "wollaston": {
            "type": "WollastonPrism",
            "properties": {"beam": "o", "transmission_ratio": em_gain},
            "tag": "internal",
        },
    },
    "order": [
        "parang_rot", "m3", "alt_rot", "lp", "hwp",
        "image_rotator", "optics", "flc", "dichroic", "wollaston"
    ],
}

system_MM = generate_system_mueller_matrix(system_dict)
print(type(system_MM))

# Starting guess for parameters (aligned with bounds)
starting_guess = np.array([
    0,  # dichroic_phi
    0 # dichroic_theta
])

# Bounds for the parameters
bounds = [
    (0, 1),          # dichroic_phi
    (-180, 180),       # dichroic_theta
]

# Example data and errors for testing
data = [np.array([1, 0.5, -0.3, 0])]  # Example observed Stokes vectors
errors = [np.array([0.01, 0.01, 0.01, 0.01])]  # Example measurement errors

# Define system parameters in the expected nested format
system_parameters = [
    ["dichroic", "phi"],
    ["dichroic", "theta"]
]

# Perform optimization
result = perform_optimization(
    starting_guess=starting_guess, 
    bounds=bounds, 
    configurations=[system_dict, system_dict, system_dict, system_dict],  # List of multiple system_dicts
    data=data, 
    errors=errors, 
    system_MM=system_MM,
    system_parameters=system_parameters, 
    S_in=[1, 1, 0, 0]
)


# Print the results
print("Optimization Result:")
print("Success:", result.success)
print("Optimal Parameters:", result.x)
print("Final Objective Value (Chi-Squared):", result.fun)

<class 'pyMuellerMat.MuellerMat.SystemMuellerMatrix'>
([0, 0.0, 0.03, 0, np.float64(1.5330762659414212), np.float64(2.900511664638528), np.float64(-2.368746399133097), np.float64(2.0618370726100776), np.float64(-4.99600063420726), np.float64(116.22201693145395), np.float64(0.022105949364102033), np.float64(3.267699783335079), np.float64(4.996455912303434), 0.0, 0, 'o', np.float64(1.1915271367205387)], [['parang_rot', 'pa'], ['m3', 'phi'], ['m3', 'epsilon'], ['alt_rot', 'pa'], ['lp', 'theta'], ['hwp', 'phi'], ['hwp', 'delta_theta'], ['image_rotator', 'phi'], ['image_rotator', 'delta_theta'], ['optics', 'phi'], ['optics', 'epsilon'], ['flc', 'phi'], ['flc', 'theta'], ['dichroic', 'phi'], ['dichroic', 'theta'], ['wollaston', 'beam'], ['wollaston', 'transmission_ratio']])


NameError: name 'shape' is not defined